In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('../data/Ames_Housing_Price_Data.csv', header = [0])
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Data Cleaning Part 1 
## Handling Null Values

In [2]:
#Dividing lot area into bins of interval size of 1000. (Max - min)/1000 = 214 bins
data['lot_bucket'] = pd.cut(data['LotArea'],214)


#Here we fill NA LotFrontage values with the mean value of the corresponding bin. We first calculate the mean lot 
#frontage (mean_lf), merge the 2 dataframes, and then fill the NA values with the corresponding mean value.
mean_lf = data.groupby('lot_bucket').agg({'LotFrontage': np.mean}, ignorena = True)
data_new = pd.merge(data, mean_lf, on = 'lot_bucket', how = 'left')
data_new.rename(columns = {'LotFrontage_x':'LotFrontage', 'LotFrontage_y':'mean_LotFrontage'}, inplace = True)
data_new['LotFrontage'].fillna(round(data_new['mean_LotFrontage'],1), inplace = True)


#Replace most N/A values with none and MasVnrType and Electrical with the mode.
categorical_list = ['Alley', 'BsmtQual', 'BsmtExposure', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
                    'PoolQC', 'Fence', 'MiscFeature', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'GarageQual',
                    'GarageCond']

numerical_list = ['MasVnrArea', 'BsmtFullBath','BsmtHalfBath','BsmtFinSF1','BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                  'GarageCars', 'GarageArea']

data_new['MasVnrType'].fillna(data_new['MasVnrType'].mode()[0], inplace = True)
data_new['Electrical'].fillna(data_new['Electrical'].mode()[0], inplace = True)

for el in categorical_list:
    data_new[el].fillna('None', inplace = True)

for el in numerical_list:
    data_new[el].fillna(0, inplace = True)


    
#402.88 is the ratio of LotArea/LotFrontage for all lot areas > 20,000 to fill in N/A values that did not have
#another value in their lotfrontage bucket.
data_new['LotFrontage'].fillna(data_new['LotArea']/402.88, inplace = True)


#This one house had a garage type listed but all other garage features were NaN
data_new.at[data_new['PID'] == 910201180, 'GarageType'] = 'None'

data_new.to_csv('Ames_Housing_Price_Data_cleaned.csv',index = False)

# Data Cleaning Part 2
# Merging  Housing Price Data, Real Estate Data, and Geo Location
At this point, the only necessary data from the real_estate_data is the address.
We will merge that data with the housing price data, along with long lat data for each house

In [3]:
housing_price_data = pd.read_csv('Ames_Housing_Price_Data_cleaned.csv')
real_estate_data = pd.read_csv('../data/Ames_Real_Estate_Data.csv')

/Users/charlesphillips/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#We are dropping the rows we deemed unnecessary from real estate data
real_estate_data = real_estate_data.drop(['Tier',
'Range','ZngCdPr','ZngCdSc','ZngOLPr','ZngOLSc','ClassPr_S','ClassSc_S','Legal_Pr','SchD_S',
'TxD_S','MA_Ownr1','MA_Ownr2','MA_Line1','MA_Line2','MA_City','MA_State','MA_Zip1','MA_Zip2','Rcrd_Yr','Rcrd_Mo','Inst1_No','Inst1_Yr',
'Inst1_Mo','Inst1TPr','LndAc_S','X1TPr_D','X1TSc_D','X2TPr_D','X2TSc_D','X1TPr_S','X1TSc_S',
'X2TPr_S','X2TSc_S','X2TSc_S'],axis = 1)

In [5]:
#This dataframe holds the merging column, and the data we need associated with
#that column
necessary_info = real_estate_data[['MapRefNo','Prop_Addr']]


In [6]:
#We are merging the two datasets by PID (in housing price data) 
#and MapRefNo (necessary_info)

housing_price_data = housing_price_data.merge(necessary_info, how = 'left',\
                                              left_on= 'PID', right_on='MapRefNo')

In [7]:
# Now we are going to merge the housing_price dataset with the geo_location of each house
# geo_data = pd.read_csv('../charles/data/geo/neighborhood_location_data.csv')
geo_data = pd.read_csv('../data/address_lat_long.csv', index_col = False)
geo_data = geo_data.drop('Unnamed: 0',axis = 1)

housing_price_data = housing_price_data.merge(geo_data, how = 'left',\
                         left_on ='Prop_Addr', right_on = 'address')
housing_price_data = housing_price_data.drop(['Unnamed: 0','address','MapRefNo'],axis = 1)
housing_price_data.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,lot_bucket,mean_LotFrontage,Prop_Addr,lat,long
0,909176150,856,126000,30,RL,64.9,7890,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,None,None,None,0,3,2010,WD,Normal,"(7298.458, 8298.201]",64.93913,436 HAYWARD AVE,42.017780,-93.651452
1,905476230,1049,139500,120,RL,42.0,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal,"(3299.486, 4299.229]",46.15493,3416 WEST ST,42.024697,-93.664186
2,905476230,1049,139500,120,RL,42.0,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal,"(3299.486, 4299.229]",46.15493,3416 WEST ST,42.024697,-93.664186
3,911128020,1001,124900,30,C (all),60.0,6060,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,None,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,None,None,None,0,11,2007,WD,Normal,"(5298.972, 6298.715]",54.06015,320 S 2ND ST,42.021389,-93.614855
4,535377150,1039,114000,70,RL,80.0,8146,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,None,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,None,None,None,0,5,2009,WD,Normal,"(7298.458, 8298.201]",64.93913,1524 DOUGLAS AVE,42.038070,-93.612065


In [8]:
housing_price_data.to_csv('Ames_Housing_Price_Data_cleaned_2.csv',index = False)

potentially drop pool, 3ssnporch

# Data Cleaning Part 3 
## Dropping Columns deemed not necessary

In [9]:
#PID was only useful in merging with the other dataset to get adresses
# housing_price_data.drop('PID',axis = 1,inplace = True)

# Street was only necessary to get lat and long, no longer necessary
housing_price_data.drop('Street',axis = 1, inplace = True)

#The following loop drops all variables that had low variability wrt count

columns_to_drop = ['Alley','LandContour','Utilities','LotConfig','LandSlope','Condition1', 
                  'Condition2', 'RoofStyle', 'RoofMatl', 'Exterior2nd', 'BsmtFinType2', 'Heating',
                  'KitchenAbvGr','Functional', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature',
                  'MiscVal', 'SaleType', 'SaleCondition']
for c in columns_to_drop:
    housing_price_data.drop(c,axis = 1,inplace = True)
    



## Combine all the Porch Data Columns


In [10]:
# Here we combine all the porch dsata into one variable
housing_price_data['TotalPorchSF'] = housing_price_data.OpenPorchSF + \
                                    housing_price_data.EnclosedPorch + \
                                    housing_price_data['3SsnPorch'] + \
                                    housing_price_data.ScreenPorch

housing_price_data.drop(['OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch'],axis = 1,inplace = True)

## Turn Select Columns into Binary Categories

In [11]:
# 1 if Reg, 0 if not Reg
data['LotShape'] = data['LotShape'].map(lambda x: 1 if x == 'Reg' else 0)

# 1 if SBrkr, 0 if not 'SBrkr'
data['Electrical'] = data['Electrical'].map(lambda x : 1 if x =='SBrkr' else 0)


In [12]:
housing_price_data.to_csv('Ames_Housing_Price_Data_cleaned_3.csv',index = False)